# Neural Affect Style Transfer

In [ ]:
import json

from numpy import zeros, concatenate, asarray, ones, amax, argmax, squeeze
from IPython.display import display, HTML
from datetime import datetime

In [ ]:
def browser_alert(message):
    display(HTML('<script type="text/javascript">alert("' + message + '");</script>'))
    
def browser_notify(message):
    display(HTML('<script type="text/javascript">var notification=new Notification("' + \
                 'Jupyter Notification",{icon:"http://blog.jupyter.org/content/' + \
                 'images/2015/02/jupyter-sq-text.png",body:"' + message + \
                 '"});</script>'))

In [ ]:
browser_notify("test")

## Read Data

In [ ]:
dataset_path = "/home/v2john/attr-reviews-dataset/dev.txt"

In [ ]:
all_texts = list()
with open(dataset_path) as dataset_file:
    for (line, text) in enumerate(dataset_file):
        text = text.split('\t')[3]
        all_texts.append(text)
        
        if line == 4999:
            break

In [ ]:
len(all_texts)

### Tokenize and build embeddings

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
keras_tokenizer = Tokenizer(num_words=10000)

In [ ]:
keras_tokenizer.fit_on_texts(all_texts)

In [ ]:
VOCAB_SIZE = len(keras_tokenizer.word_index) + 1

In [ ]:
VOCAB_SIZE

In [ ]:
text_sequences = keras_tokenizer.texts_to_sequences(all_texts)

In [ ]:
text_sequences = asarray(text_sequences)

In [ ]:
text_sequences.shape

## Build pre-trained embeddings

In [ ]:
from gensim.models.wrappers import fasttext
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
embeddings_path = "/home/v2john/pretrained-embeddings/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
w2v_model = KeyedVectors.load_word2vec_format(embeddings_path, binary=True, unicode_errors='ignore')

In [ ]:
browser_notify("Embeddings loaded")

In [ ]:
EMBEDDING_DIM = squeeze(w2v_model.word_vec("the").shape)

In [ ]:
EMBEDDING_DIM

In [ ]:
embeddings_matrix = zeros(shape=(len(keras_tokenizer.word_index) + 1, EMBEDDING_DIM))

In [ ]:
for word in keras_tokenizer.word_index:
    embeddings_matrix[keras_tokenizer.word_index[word]] = w2v_model.word_vec("the")

In [ ]:
embeddings_matrix.shape

## Keras Model

In [ ]:
from keras import backend as K
from keras.layers import Input, Dense, RepeatVector, LSTM, Conv1D, Masking, Embedding, Dropout, GRU
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.models import Model
from keras.losses import binary_crossentropy
from keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_SEQUENCE_LENGTH = 20

In [ ]:
padded_text_sequences = \
    pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', 
                  truncating='post', value=0)

In [ ]:
padded_text_sequences.shape

In [ ]:
word_dict = dict()
for word in keras_tokenizer.word_index:
    rank = keras_tokenizer.word_index[word]
    if rank <= VOCAB_SIZE:
        word_dict[rank] = word

In [ ]:
x_train = zeros(shape=(len(padded_text_sequences), MAX_SEQUENCE_LENGTH, VOCAB_SIZE))

In [ ]:
for i in range(len(padded_text_sequences)):
    for pos in range(len(padded_text_sequences[i])):
        x_train[i][pos][padded_text_sequences[i][pos]] = 1

In [ ]:
x_train.shape

In [ ]:
main_input = Input(shape=padded_text_sequences.shape[1:], dtype='float32', name='main_input')
print(main_input)

embed_1 = Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embeddings_matrix],
                    input_length=MAX_SEQUENCE_LENGTH, trainable=False, mask_zero=True) (main_input)
print(embed_1)

# lstm_1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.25, 
#                             recurrent_dropout=0.25, name='lstm_1')) (embed_1)
# print(lstm_1)

lstm_2 = Bidirectional(GRU(512, name='lstm_2'))(embed_1)
print(lstm_2)

repeat_1 = RepeatVector(MAX_SEQUENCE_LENGTH, name='repeat_1')(lstm_2)
print(repeat_1)

lstm_3 = Bidirectional(GRU(512, return_sequences=True, name='lstm_3'))(repeat_1)
print(lstm_3)

lstm_4 = Bidirectional(GRU(512, return_sequences=True, dropout=0.25, 
                            recurrent_dropout=0.25, name='lstm_4'))(lstm_3)
print(lstm_4)

lstm_5 = Bidirectional(GRU(512, return_sequences=True, dropout=0.25, 
                            recurrent_dropout=0.25, name='lstm_5'))(lstm_4)
print(lstm_5)

output = TimeDistributed(Dense(VOCAB_SIZE, activation="softmax")) (lstm_5)
print(output)

In [ ]:
model = Model(main_input, output)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(padded_text_sequences, x_train, batch_size=512, epochs=100, verbose=1)

In [ ]:
predictions = model.predict(padded_text_sequences)

In [ ]:
predictions.shape

In [ ]:
browser_notify("Model Trained")

In [ ]:
MAX_PROB = 0.2

In [ ]:
word_dict = dict()
for word in keras_tokenizer.word_index:
    rank = keras_tokenizer.word_index[word]
    if rank <= VOCAB_SIZE:
        word_dict[rank] = word

In [ ]:
def sequence_to_str(sequence):
    word_list = list()
    for element in sequence:
#         if amax(element) < MAX_PROB:
#             continue        
        index = argmax(element) + 1
        word = word_dict[index]
        word_list.append(word)
        
    return word_list

In [ ]:
predictions_file_path = "/home/v2john/generated_sentences/" + datetime.now().strftime('%Y-%m-%d-%H-%M-%S') + ".txt"

In [ ]:
with open(predictions_file_path, 'w') as predictions_file:
    
    for i in range(len(predictions)):
        
        predicted_word_list = sequence_to_str(predictions[i])
        actual_len = len(all_texts[i].split())
        
        actual_sentence = all_texts[i].strip()
        generated_sentence = (" ".join(predicted_word_list[:actual_len])).strip()
        
        sent_dict = dict()
        sent_dict["actual"] = actual_sentence
        sent_dict["generated"] = generated_sentence
        
        predictions_file.write(json.dumps(sent_dict, sort_keys=True, indent=2, separators=(',', ': ')))
        predictions_file.write("\n")

In [ ]:
browser_notify("Sentences generated")